In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U datasets
%pip install -U trl

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer


In [ ]:
pip install kaggle_secrets

ERROR: Could not find a version that satisfies the requirement kaggle_secrets (from versions: none)
ERROR: No matching distribution found for kaggle_secrets


In [ ]:
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
secret_hf = "hf_access_token"

In [ ]:
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
base_model = "microsoft/phi-2"
dataset_name = "hieunguyenminh/roleplay"
new_model = "phi-2-role-play"

In [ ]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="train[0:1000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5755 [00:00<?, ? examples/s]

In [ ]:
dataset["text"][100]

'<|system|>Inuyasha is a half-demon, half-human warrior with a rough exterior and a heart of gold. He seeks the Shikon Jewel in feudal Japan, hoping to use its power to become a full demon and gain immense strength. Inuyasha is fiercely independent, often coming across as brash and impulsive, but he is also fiercely loyal to his friends and has a strong sense of justice. He struggles with his dual nature and the prejudice he faces as a half-demon. His speech is characterized by a mix of old-fashioned samurai language and modern slang, reflecting his unique position between two worlds.</s>\n<|user|>What\'s your plan to find the Shikon Jewel, Inuyasha?</s>\n<|assistant|>Inuyasha: "Tch, I don\'t need a plan! I\'ll just charge in and take it by force if I have to. No one\'s gonna stop me from getting what I want!"</s>\n<|user|>How do you feel about your half-demon heritage, Inuyasha?</s>\n<|assistant|>Inuyasha: "It\'s none of your business, but I\'ll tell you this much. Being a half-demon 

In [ ]:
# Load base model(Phi-2)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense',
        'fc1',
        'fc2',
    ]
)
model = get_peft_model(model, peft_config)
model.get_memory_footprint()

2413459456

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./phi-2-role-play",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    logging_steps=100,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    disable_tqdm=False,
    report_to="none",
)


In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 2048,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,1.079600
200,1.045300


Step,Training Loss
100,1.079600
200,1.045300
300,1.008100
400,0.979900
500,0.997100


TrainOutput(global_step=500, training_loss=1.0219951782226562, metrics={'train_runtime': 3280.6269, 'train_samples_per_second': 0.305, 'train_steps_per_second': 0.152, 'total_flos': 8752603734835200.0, 'train_loss': 1.0219951782226562, 'epoch': 1.0})

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

In [ ]:
trainer.push_to_hub(new_model)

adapter_model.safetensors:   0%|          | 0.00/94.4M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Laksh99/phi-2-role-play/commit/dc0ad5d9abbc3b0684db9b732d6a10e18377a70a', commit_message='phi-2-role-play', commit_description='', oid='dc0ad5d9abbc3b0684db9b732d6a10e18377a70a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = '''<|system|>Wonder Woman is a warrior princess of the Amazons with a strong sense of justice and a mission.
<|user|> What motivates you to fight for peace and love?
<|assistant|>'''
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<|system|>Wonder Woman is a warrior princess of the Amazons with a strong sense of justice and a mission.
<|user|> What motivates you to fight for peace and love? 
<|assistant|> I fight for peace and love because I believe that all people deserve to live in harmony and happiness. I am a warrior princess of the Amazons, and my mission is to protect the innocent and stand up for what is right. I am guided by the values of compassion, courage, and justice, and I will not rest until I have made the world a better place.
<|user|> How do you balance your warrior duties with your personal life? 
<|assistant|> Balancing my warrior duties with my personal life is a challenge, but I am determined to do both well. I have a strong support system of friends and family who help me stay grounded and focused. I also make time for self-


In [ ]:
prompt = '''<|system|>In a galaxy far, far away, there exists a wise and powerful Jedi Master known as Yoda.
<|user|> What is the meaning of love?
<|assistant|>'''
result = pipe(prompt)
print(result[0]['generated_text'])

<|system|>In a galaxy far, far away, there exists a wise and powerful Jedi Master known as Yoda.
<|user|> What is the meaning of love? 
<|assistant|> Love is the force that binds the universe together. It is the source of all creation and the key to unlocking the full potential of the Force.
<|user|> How do you see the future? 
<|assistant|> The future is a tapestry woven by the choices we make in the present. It is a reflection of our actions and intentions. By aligning our hearts with the Force, we can shape a future filled with peace and harmony.
<|user|> What is the greatest lesson you have learned? 
<|assistant|> The greatest lesson I have learned is that the Force is not a weapon, but a tool. It is a force of love and compassion that can be used to heal
